In [4]:
from scripts import facebook_model, linkedin_model
import pandas as pd

df = pd.read_csv('data_linkedin/linkedin_data.csv')
text = df['POSTS'][233]
reacts = df['INDEX'][233]

print(reacts)

result = linkedin_model.get_index_prediction(text)
print(result)


1.0780073836122166


ValueError: np.nan is an invalid document, expected byte or unicode string.

In [3]:
df = pd.read_csv('data_facebook/facebook_data.csv')
text = df['POSTS'][233]
reacts = df['ALL_REACTIONS'][233]

print(reacts)

result = facebook_model.get_reactions_prediction(text)
print(result)


19
19
